Data: https://opus.nlpl.eu/opus-100.php

In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass
from torch.utils.data import DataLoader, Dataset

In [2]:
@dataclass
class paths:
    data = os.path.join('data', 'en-pl')
    pl_test = os.path.join(data, 'opus.en-pl-test.pl')
    en_test = os.path.join(data, 'opus.en-pl-test.en')
    pl_dev = os.path.join(data, 'opus.en-pl-dev.pl')
    en_dev = os.path.join(data, 'opus.en-pl-dev.en')
    pl_train = os.path.join(data, 'opus.en-pl-train.pl')
    en_train = os.path.join(data, 'opus.en-pl-train.en')

In [3]:
with open(paths.pl_train, 'r', encoding='UTF-8') as f:
    pl_data = f.read()
    pl_data = pl_data.split('\n')[:-1]

In [4]:
with open(paths.en_train, 'r', encoding='UTF-8') as f:
    en_data = f.read()
    en_data = en_data.split('\n')[:-1]

In [5]:
pd.DataFrame(data={
    'polish': pl_data, 
    'english': en_data
})

,polish,english
0,W budynku są intruzi!,Intruders in the building!
1,Bezmyślność...,Very careless of me.
2,"KOMISJA WSPÓLNOT EUROPEJSKICH,","THE COMMISSION OF THE EUROPEAN COMMUNITIES,"
3,Pewnie.,"- Yeah, right."
4,Nie!,No!
...,...,...
999995,Uciskaj dalej.,Keep pressure on.
999996,Kto tu jeszcze był?,Who else was here?
999997,"Chwilowo nie wiemy, gdzie się znajduje. Nadal ...","It's been temporarily misplaced, your honor, b..."
999998,Mam syna.,I have a son.
